In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
# Trying to convert the input data, which starts off as dataframeX, must take 5 days and then transpose it
# and then convert it into a numpy array

rawdata = pd.read_csv('data/bitcoin.csv',usecols=[8])
days = 5                                                      # Number of days we are looking at
predict_delta = 7                                             # Number of days we are predicting into the future 
number_of_items = rawdata.shape[0] - (predict_delta + days)   # number of tests
print(rawdata.shape)

X = []                                                        # five day blocks in a list
# For every index in x, there is a y that holds the actual 
# value that we are trying to predict (seven days into the future)
Y = []

# Splits the array of closing prices up into multiple arrays, each holding five days worth of prices
# then transposes each array and finally appends them onto one large matrix
for i in range(number_of_items):
    X.append(rawdata.head(days + i).tail(days).as_matrix().transpose().tolist()[0])
    predict_day = rawdata.head(days + i + predict_delta).tail(1).as_matrix().transpose().tolist()[0][0]
    #change = (predict_day - X[i][4])/predict_day
    Y.append((predict_day - X[i][4])/predict_day)
    #if change >= .03:
        #Y.append(np.array([[1,0,0]]))
    #elif change < .03 and change > -.03:
        #Y.append(np.array([[0,1,0]]))
    #else:
        #Y.append(np.array([[0,0,1]]))
# converts python arrays into numpy arrays
X = np.array(X)
Y = np.array(Y)


(1745, 1)


In [14]:
model = Sequential()
model.add(Dense(1,input_shape=(5,)))
model.add(Activation('relu'))
model.add(Dense(1, activation='tanh'))
model.add(Dropout(2.0))

model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

H = model.fit(X, Y, epochs=5)

Epoch 1/5
1733/1733 [==============================] - 0s 163us/step - loss: 0.1779 - acc: 5.7703e-04
Epoch 2/5
1733/1733 [==============================] - 0s 82us/step - loss: 0.1779 - acc: 5.7703e-04
Epoch 3/5
1733/1733 [==============================] - 0s 81us/step - loss: 0.1779 - acc: 5.7703e-04
Epoch 4/5
1733/1733 [==============================] - 0s 88us/step - loss: 0.1779 - acc: 5.7703e-04
Epoch 5/5
1733/1733 [==============================] - 0s 80us/step - loss: 0.1779 - acc: 5.7703e-04


In [12]:
XVal = np.array([[8070.8, 8891.21, 8516.24, 9477.84, 10016.49]])
result = model.predict(XVal)
result = result[0][0]

if result >= 1 - 2/3:
    print("it will go up by 3%")
elif result > 0 - 2/3 and result < 1 - 2/3:
    print("it will stay between 3%")
else:
    print("it will go down by 3%")

print(result)

it will stay between 3%
0.0
